# 데커레이터

## 데커레이터는 함수가 함수를 한번 감싸서 리턴하는 형태이다.
### 1번과 2번은 동일하다

In [2]:
def decorate(func) :
    return func

#1
@decorate
def target() :
    print("running target()")
target()

running target()


In [3]:
    
#2
def target() :
    print("running target()")
target = decorate(target)
target()

running target()


## 데커레이터가 실행되는 시점
### 데커레이터는 import time 에 실행되며 이는 runtime 과 구분된다.

In [9]:
# registration.py 을 확인하고 콘솔에서 실행해보자
import registration

In [10]:
# import time에 데코레이트 함수가 실행되어 registry에 두 함수가 추가된 것을 확인할 수 있다.
registration.registry

[<function registration.f1()>, <function registration.f2()>]

## 데커레이터로 개선된 전략 패턴
### promos 에 리스트를 추가하는 것을 깜빡하였을 경우를 데커레이터를 활용해 해결한다.

In [ ]:
promos = []

# 데커레이트된 함수들은 import time에 실행되어 promos 에 추가될 것이다.
def promotion(promo_func) :
    promos.append(promo_func)
    return promo_func

@promotion
def fidelity_promo(order) :
    """충성도 포인트가 1000점 이상인 고객에게 전체 5% 할인 적용"""
    return order.total() * 0.05 if order.customer.fidelity >= 1000 else 0
@promotion
def bulk_item_promo(order) :
    """20개 이상의 동일 상품을 구입하면 10% 할인 적용"""
    discount = 0
    for item in order.cart :
        if item.quantity >= 20 :
            discount += item.total() * 0.1
    return discount

@promotion
def large_order_promo(order) :
    '''10종류 이상의 상품을 구입하면 전체 7% 할인 적용'''
    distinct_items = {item.product for item in order.cart}
    if len(distinct_items) >= 10 :
        return order.total() * .07
    return 0

def best_promo(order) :
    """ 최대로 할인받을 금액을 반환한다."""
    return max(promo(order) for promo in promos)

## 아래 코드가 에러가 나는 이유
### f1() 은 b를 전역변수라고 본다. 
### f2() 는 b를 지역변수라고 본다. b가 함수 내에서 선언되었기 때문이다.
### 따라서 f2()는 b를 선언 전에 참조한 것이고 이는 에러가 발생한다.

In [14]:
b = 3
def f1(a) :
    print(a)
    print(b)
f1(3)

3
3


In [15]:
b = 6
def f2(a) :
    print(a)
    print(b)
    b=6
f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

In [17]:
from dis import dis
# b를 전역변수로 간주하고 있다.
dis(f1)

  3           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  4           8 LOAD_GLOBAL              0 (print)
             10 LOAD_GLOBAL              1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP
             16 LOAD_CONST               0 (None)
             18 RETURN_VALUE


In [18]:
# b를 지역변수로 간주하고 있다.
dis(f2)

  3           0 LOAD_GLOBAL              0 (print)
              2 LOAD_FAST                0 (a)
              4 CALL_FUNCTION            1
              6 POP_TOP

  4           8 LOAD_GLOBAL              0 (print)
             10 LOAD_FAST                1 (b)
             12 CALL_FUNCTION            1
             14 POP_TOP

  5          16 LOAD_CONST               1 (6)
             18 STORE_FAST               1 (b)
             20 LOAD_CONST               0 (None)
             22 RETURN_VALUE


# 클로저
### 대부분의 데커레이터는 데커레이트된 함수를 변경한다
### 즉 내부 함수를 정의하고 그것을 반환하여 데커레이트된 함수를 대체한다.
### 내부 함수를 사용하는 코드는 제대로 작동하기 위해 거의 항상 클로저에 의존한다.

## 클로저는 함수 본체 외부에 정의된 비전역 변수에 접근할 수 있다.

In [19]:
class Averager() :
    def __init__(self) :
        self.series = []
        
    def __call__(self, new_value) :
        self.series.append(new_value)
        total = sum(self.series)
        return total/len(self.series)
    
avg = Averager()
print(avg(10))
print(avg(11))
print(avg(12))

10.0
10.5
11.0


In [20]:
def make_averager() :
    series = []
    
    def averager(new_value) :
        series.append(new_value)
        total = sum(series)
        return total/len(series)
    
    return averager

avg = make_averager()
print(avg(10))
print(avg(11))
print(avg(12))

10.0
10.5
11.0


### Averager class 는 Averager.series 에 값들이 저장되어 있다.
### 하지만 make_averager() 어디에 series 가 저장되어 있는 걸까
### make_averager()는 이미 함수를 반환했기 때문에 지역 범위도 사라졌다.
### 이때 series를 자유변수라 하고 클로저는 이 자유변수에 접근할 수 있다.
![image](images/closure.PNG)

In [21]:
avg.__code__.co_varnames

('new_value', 'total')

In [23]:
# series는 자유변수에 속한다.
avg.__code__.co_freevars

('series',)

In [32]:
avg.__closure__[0].cell_contents

[10, 11, 12]

# nonlocal
### 함수 외부에서 정의된 자유변수에 접근 할 수 있게 해준다.

In [34]:
# f2() 의 경우와 마찬가지로 count가 averager 함수 내에서 선언되었으므로 지역변수로 봤고
# count 선언되기 전에 참조하므로 오류가 발생한다.

def make_averager() :
    count = 0
    total = 0
    
    def averager(new_value) :
        count += 1
        total += new_value
        return total/count
    
    return averager

avg = make_averager()
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

In [35]:
# 이는 다음과 같이 해결할 수 있다.
def make_averager() :
    count = 0
    total = 0
    
    def averager(new_value) :
        nonlocal count, total
        count += 1
        total += new_value
        return total/count
    
    return averager

avg = make_averager()
avg(10)

10.0